In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import scipy.stats as ss
from sklearn.metrics import classification_report

In [2]:
compas = pd.read_csv('../data/compas-scores-two-years.csv', encoding='latin-1')

In [3]:
# Filter data for useful rows
compas = compas[compas['days_b_screening_arrest'] >= -30]
compas = compas[compas['days_b_screening_arrest'] <= 30]
compas = compas[compas['is_recid'] != -1]
compas = compas[compas['c_charge_degree'] != "O"]
compas = compas[compas['score_text'] != "N/A"]
compas.head()

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,1,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0
1,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,1,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1
2,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,...,3,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1
5,7,marsha miles,marsha,miles,2013-11-30,Male,1971-08-22,44,25 - 45,Other,...,1,Low,2013-11-30,2013-11-30,2013-12-01,0,1,853,0,0
6,8,edward riddle,edward,riddle,2014-02-19,Male,1974-07-23,41,25 - 45,Caucasian,...,2,Low,2014-02-19,2014-03-31,2014-04-18,14,5,40,1,1


In [4]:
compas = compas[["sex","age","race","decile_score","priors_count",
                 "c_charge_degree","two_year_recid","c_jail_in", "c_jail_out"]]

compas = compas.loc[compas.race.isin(["Caucasian", "African-American"])]

In [5]:
# Convert features to categorical type
compas_encoded = compas.copy()
compas_encoded.sex = pd.get_dummies(compas["sex"])["Female"]
compas_encoded.race = pd.get_dummies(compas["race"])["African-American"]
compas_encoded.c_charge_degree = pd.get_dummies(compas["c_charge_degree"])["F"]

# Calculate length of stay to use in model (log hours)
compas_encoded['c_jail_in'] = pd.to_datetime(compas_encoded['c_jail_in'])
compas_encoded['c_jail_out'] = pd.to_datetime(compas_encoded['c_jail_out'])
compas_encoded['los'] = np.log((compas_encoded['c_jail_out']-compas_encoded['c_jail_in']).astype('timedelta64[h]'))

compas_encoded.drop(["c_jail_in", "c_jail_out"], axis = 1, inplace=True)

compas_encoded.replace([np.inf, -np.inf], np.nan, inplace=True)
compas_encoded.dropna(inplace=True)

/Users/sarahkurihara/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/sarahkurihara/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [6]:
compas_encoded.head()
compas_encoded.drop_duplicates(keep = "first", inplace = True)

In [7]:
# X = features, S = sensitive attributes, E = explanatory attribute, Y = response
X = ['sex','age','decile_score','priors_count', 'los']
S = "race"
E = "c_charge_degree"
Y = "two_year_recid"

X_ALL = ['sex','age','decile_score','priors_count','race','c_charge_degree','los']

In [8]:
# 5:1:1 split
np.random.seed(5243)
train, test = train_test_split(compas_encoded, test_size=1/7)
train, val = train_test_split(train, test_size=1/6)

In [9]:
print(train.shape, test.shape, val.shape)

(3610, 8) (723, 8) (723, 8)


In [10]:
X_train = train[X_ALL]
L_train = train[X]
s_train = train[S]
e_train = train[E]
y_train = train[Y]

X_val = val[X_ALL]
L_val = val[X]
s_val = val[S]
e_val = val[E]
y_val = val[Y]

X_test = test[X_ALL]
L_test = test[X]
s_test = test[S]
e_test = test[E]
y_test = test[Y]

In [11]:
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
clf.score(X_val, y_val)

0.686030428769018

## Create functions used in pseudocode

In [12]:
# Creates a list of partitions: 1 for each unique value of e

# X is the full dataset (in our case, train)
# e is the 
def PARTITION(X):
    partitions = list()
    
    for e_i in np.unique(X[E]):
        partitions.append(X[X[E]==e_i])
    
    return partitions

In [13]:
# Delta function returns the number of observations (i.e. people) who are incorrectly classified 
# based on theoretical probabilities of reciding, calculated as the average rate of reciding
# for each explanatory varaible (in our case, type of crime comittied, c_charge_degree)

def DELTA(X, X_ei, s_i):
    
    # Gi is the number of observations for each race
    # Don't we need to pass S as a parameter for this function?
    Gi = sum(X_ei[S] == s_i)
    
    # X_ei_si is the dataset that contains the observations for each race
    X_ei_si = X_ei[X_ei[S] == s_i]
    
    # P_denom is the number of people in group 
    # P_num is number of observations who recid
    P_denom = X_ei_si.shape[0]
    P_num = sum(X_ei_si[Y] == 1)
    
    # P is the probability of reciding for one race
    # It is calculated by taking number of people who recid in each group 
    # dividied by total number of people in that group
    P = P_num/P_denom
    
    # All other observations (for the other group)
    X_ei_not_si = X_ei[X_ei[S] != s_i]
    
    # The probability of reciding for the other group (same calculation as above)
    Ps_2 = sum(X_ei_not_si[Y] == 1)/X_ei_not_si.shape[0]
    
    # Ps is P*, which is the theoretical true probability of reciding
    # Calculated by the average 
    Ps = (P+Ps_2)/2
    
    # Calcualte the number of incorrectly classified people
    d = int(round(Gi * abs(P - Ps)))
    
    return(d)

# Local Massaging

In [14]:
relabeled_X_ei = list()

for X_ei in PARTITION(train):
    X_ei_copy = X_ei.copy()
    
    ranker_model = LogisticRegression(random_state=0).fit(X_ei[X_ALL], X_ei[Y])
    
    afam_index = [i for (i, v) in zip(list(range(X_ei.shape[0])), list(X_ei[S] == 1)) if v]
    afam = X_ei[X_ei[S] == 1].copy()
    delta_afam = DELTA(train, X_ei, 1)
    afam_predicted_1_index = [afam_index[v] for v in np.squeeze(np.where(ranker_model.predict(afam[X_ALL]) == 1))]
    afam_predicted_1_index_Y1 = [i for (i,v) in zip(afam_predicted_1_index, X_ei.iloc[afam_predicted_1_index][Y]) if v==1]
    afam_predicted_1 = X_ei.iloc[afam_predicted_1_index_Y1]
    
    afam_ranks = (ss.rankdata(ranker_model.decision_function(afam_predicted_1[X_ALL]))-1).astype(int)
    afam_tochange = [i for (i, v) in zip(list(range(len(afam_ranks))), afam_ranks < delta_afam) if v]
    afam_tochange_idx = [afam_predicted_1_index_Y1[v] for v in afam_tochange]
    
    cauca_index = [i for (i, v) in zip(list(range(X_ei.shape[0])), list(X_ei[S] == 0)) if v]
    cauca = X_ei[X_ei[S] == 0].copy()
    delta_cauca = DELTA(train, X_ei, 0)
    cauca_predicted_0_index = [cauca_index[v] for v in np.squeeze(np.where(ranker_model.predict(cauca[X_ALL]) == 0))]
    cauca_predicted_0_index_Y0 = [i for (i,v) in zip(cauca_predicted_0_index, X_ei.iloc[cauca_predicted_0_index][Y]) if v==0]
    cauca_predicted_0 = X_ei.iloc[cauca_predicted_0_index_Y0]
    
    cauca_ranks = (ss.rankdata(-ranker_model.decision_function(cauca_predicted_0[X_ALL]))-1).astype(int)
    cauca_tochange = [i for (i, v) in zip(list(range(len(cauca_ranks))), cauca_ranks < delta_cauca) if v]
    cauca_tochange_idx = [cauca_predicted_0_index_Y0[v] for v in cauca_tochange]
    
    for i in afam_tochange_idx:
        X_ei_copy.loc[X_ei_copy.index[i], Y] = 0
    for i in cauca_tochange_idx:
        X_ei_copy.loc[X_ei_copy.index[i], Y] = 1
    
    relabeled_X_ei.append(X_ei_copy)
    
    print("DELTA(African American) = ", delta_afam, "African Americans changed from 1 to 0")
    print("DELTA(Caucasian) = ", delta_cauca, "Caucasians changed from 0 to 1")
    
local_massaging = pd.concat(relabeled_X_ei)

DELTA(African American) =  44 African Americans changed from 1 to 0
DELTA(Caucasian) =  38 Caucasians changed from 0 to 1
DELTA(African American) =  99 African Americans changed from 1 to 0
DELTA(Caucasian) =  57 Caucasians changed from 0 to 1


In [15]:
lm_X_train = local_massaging[X_ALL]
lm_Y_train = local_massaging[Y]

In [16]:
clf = LogisticRegression(random_state=0).fit(lm_X_train, lm_Y_train)
clf.score(X_val[X_ALL], y_val)

0.6708160442600276

In [17]:
# Total number changed values should be sum of all DELTAs shown above
res = [1 for i, j in zip(train.sort_index()["two_year_recid"], pd.DataFrame(lm_Y_train).sort_index()["two_year_recid"]) if i != j]
sum(res)

238

# Evaluation

Notation: P_c stands for probability based on the classifier's predictions.

### Parity or D_all

Parity is defined as the difference is positive prediction rates in the two race groups. Paper 6 also calls this D_all, which stands for all discrimination. Fairness calls for Parity being close to 0.

<br>
<center>Parity = |P_c(recid = 1 | race = African American) - P_c(recid = 1 | race = Caucasian)</center>
    
### Calibration

Calibration is defined as the difference in accuracies between the two race groups. Fairness calls for Calibration being close to 0.

<br>

<center>Calibration = |P_c(recid predicted correctly | race = African American) - P_c(recid predicted correctly | race = Caucasian)</center>

### Equality of Odds

Equality of odds is achieved when the difference in positive prediction rates is equal for the two race groups. Fairness calls for the following value to be close to 0 for both y in {0,1}.

<br>

<center>D_Odds = P_c(recid.hat = 1 | race = African American, recid = y) - P_c(recid.hat = 1 | race = Caucasian, recid = y)</center>



In [18]:
# X must include the sensitive feature
def PARITY(X, Y_PRED):
    s = X[S]
    
    afam = X[X[S] == 1]
    num_afam = sum(Y_PRED[X[S] == 1])
    den_afam = afam.shape[0]
    
    cauca = X[X[S] == 0]
    num_cauca = sum(Y_PRED[X[S] == 0])
    den_cauca = cauca.shape[0]
    
    print("P_c(recid = 1 | race = African American) =", num_afam/den_afam)
    print("P_c(recid = 1 | race = Caucasian) =", num_cauca/den_cauca)
    parity = abs(num_afam/den_afam - num_cauca/den_cauca)
    print("Parity =", parity)
    
    return(parity)

In [19]:
# X must include S
def CALIBRATION(X, Y_TRUE, Y_PRED):
    
    afam = X[X[S] == 1]
    Y_TRUE_afam = Y_TRUE[X[S] == 1]
    num_afam = sum([1 for (i, v) in zip(Y_TRUE_afam, Y_PRED[X[S]==1]) if i == v])
    den_afam = afam.shape[0]
    
    cauca = X[X[S] == 0]
    Y_TRUE_cauca = Y_TRUE[X[S] == 0]
    num_cauca = sum([1 for (i, v) in zip(Y_TRUE_cauca, Y_PRED[X[S]==0]) if i == v])
    den_cauca = cauca.shape[0]
    
    print("P_c(recid predicted correctly | race = African American) =", num_afam/den_afam)
    print("P_c(recid predicted correctly | race = Caucasian) =", num_cauca/den_cauca)
    calibration = abs(num_afam/den_afam - num_cauca/den_cauca)
    print("Calibration =", calibration)

In [20]:
def EQUALITY_OF_ODDS(X, Y_TRUE, Y_PRED):
    
    # S = afam, Y = 0
    X_afam_0 = X[np.logical_and(X[S]==1, Y_TRUE == 0)]
    Y_PRED_afam_0 = Y_PRED[np.logical_and(X[S]==1, Y_TRUE == 0)]
    num_afam_0 = sum([1 for i in Y_PRED_afam_0 if i == 1])
    denom_afam_0 = X_afam_0.shape[0]
    P_afam_0 = num_afam_0/denom_afam_0
    
    # S = afam, Y = 1
    X_afam_1 = X[np.logical_and(X[S]==1, Y_TRUE == 1)]
    Y_PRED_afam_1 = Y_PRED[np.logical_and(X[S]==1, Y_TRUE == 1)]
    num_afam_1 = sum([1 for i in Y_PRED_afam_1 if i == 1])
    denom_afam_1 = X_afam_1.shape[0]
    P_afam_1 = num_afam_1/denom_afam_1
    
    # S = cauca, Y = 0
    X_cauca_0 = X[np.logical_and(X[S]==0, Y_TRUE == 0)]
    Y_PRED_cauca_0 = Y_PRED[np.logical_and(X[S]==0, Y_TRUE == 0)]
    num_cauca_0 = sum([1 for i in Y_PRED_cauca_0 if i == 1])
    denom_cauca_0 = X_cauca_0.shape[0]
    P_cauca_0 = num_cauca_0/denom_cauca_0
    
    # S = cauca, Y = 1
    X_cauca_1 = X[np.logical_and(X[S]==0, Y_TRUE == 1)]
    Y_PRED_cauca_1 = Y_PRED[np.logical_and(X[S]==0, Y_TRUE == 1)]
    num_cauca_1 = sum([1 for i in Y_PRED_cauca_1 if i == 1])
    denom_cauca_1 = X_cauca_1.shape[0]
    P_cauca_1 = num_cauca_1/denom_cauca_1
    
    print("For recid = 0:\n")
    print("P_c(recid.hat = 1 | race = African American, recid = 0) = ", P_afam_0)
    print("P_c(recid.hat = 1 | race = Caucasian, recid = 0) = ", P_cauca_0)
    print("Difference in odds of true recid = 0 is  = D_FPR =", abs(P_afam_0 - P_cauca_0))
    print("\n\n")
    print("For recid = 1:\n")
    print("P_c(recid.hat = 1 | race = African American, recid = 1) = ", P_afam_1)
    print("P_c(recid.hat = 1 | race = Caucasian, recid = 1) = ", P_cauca_1)
    print("Difference in odds of true recid = 1 is = D_TPR =", abs(P_afam_1 - P_cauca_1))
    


In [21]:
def D_FNR(X, Y_TRUE, Y_PRED):
    # S = afam, Y = 1
    X_afam_1 = X[np.logical_and(X[S]==1, Y_TRUE == 1)]
    Y_PRED_afam_1 = Y_PRED[np.logical_and(X[S]==1, Y_TRUE == 1)]
    num_afam_1 = sum([1 for i in Y_PRED_afam_1 if i == 0])
    denom_afam_1 = X_afam_1.shape[0]
    P_afam_1 = num_afam_1/denom_afam_1
    
    # S = cauca, Y = 1
    X_cauca_1 = X[np.logical_and(X[S]==0, Y_TRUE == 1)]
    Y_PRED_cauca_1 = Y_PRED[np.logical_and(X[S]==0, Y_TRUE == 1)]
    num_cauca_1 = sum([1 for i in Y_PRED_cauca_1 if i == 0])
    denom_cauca_1 = X_cauca_1.shape[0]
    P_cauca_1 = num_cauca_1/denom_cauca_1
    
    print("Difference in False Negative Rates")

    print("For recid = 1:\n")
    print("P_c(recid.hat = 0 | race = African American, recid = 1) = ", P_afam_1)
    print("P_c(recid.hat = 0 | race = Caucasian, recid = 1) = ", P_cauca_1)
    
    print("D_FNR =", abs(P_afam_1 - P_cauca_1))

## Baseline

In [22]:
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
baseline_pred = clf.predict(X_val[X_ALL])
clf.score(X_val, y_val)

0.686030428769018

In [23]:
print(classification_report(y_val, clf.predict(X_val[X_ALL])))

              precision    recall  f1-score   support

           0       0.68      0.75      0.71       377
           1       0.69      0.62      0.65       346

    accuracy                           0.69       723
   macro avg       0.69      0.68      0.68       723
weighted avg       0.69      0.69      0.68       723



In [24]:
# Parity

PARITY(X_val, baseline_pred)

P_c(recid = 1 | race = African American) = 0.5011286681715575
P_c(recid = 1 | race = Caucasian) = 0.31785714285714284
Parity = 0.1832715253144147


0.1832715253144147

In [25]:
# Calibration

CALIBRATION(X_val, y_val, baseline_pred)

P_c(recid predicted correctly | race = African American) = 0.6884875846501128
P_c(recid predicted correctly | race = Caucasian) = 0.6821428571428572
Calibration = 0.006344727507255676


In [26]:
# Equality of Odds

EQUALITY_OF_ODDS(X_val, y_val, baseline_pred)

For recid = 0:

P_c(recid.hat = 1 | race = African American, recid = 0) =  0.31221719457013575
P_c(recid.hat = 1 | race = Caucasian, recid = 0) =  0.17307692307692307
Difference in odds of true recid = 0 is  = D_FPR = 0.13914027149321267



For recid = 1:

P_c(recid.hat = 1 | race = African American, recid = 1) =  0.6891891891891891
P_c(recid.hat = 1 | race = Caucasian, recid = 1) =  0.5
Difference in odds of true recid = 1 is = D_TPR = 0.18918918918918914


In [27]:
# D_FNR

D_FNR(X_val, y_val, baseline_pred)

Difference in False Negative Rates
For recid = 1:

P_c(recid.hat = 0 | race = African American, recid = 1) =  0.3108108108108108
P_c(recid.hat = 0 | race = Caucasian, recid = 1) =  0.5
D_FNR = 0.1891891891891892


## Local Massaging

In [28]:
clf = LogisticRegression(random_state=0).fit(lm_X_train, lm_Y_train)
lm_pred = clf.predict(X_val[X_ALL])
clf.score(X_val[X_ALL], y_val)

0.6708160442600276

In [29]:
print(classification_report(y_val, clf.predict(X_val[X_ALL])))

              precision    recall  f1-score   support

           0       0.67      0.74      0.70       377
           1       0.68      0.60      0.63       346

    accuracy                           0.67       723
   macro avg       0.67      0.67      0.67       723
weighted avg       0.67      0.67      0.67       723



In [30]:
# Parity

PARITY(X_val, lm_pred)

P_c(recid = 1 | race = African American) = 0.3905191873589165
P_c(recid = 1 | race = Caucasian) = 0.46785714285714286
Parity = 0.07733795549822636


0.07733795549822636

In [31]:
# Calibration

CALIBRATION(X_val, y_val, lm_pred)

P_c(recid predicted correctly | race = African American) = 0.672686230248307
P_c(recid predicted correctly | race = Caucasian) = 0.6678571428571428
Calibration = 0.004829087391164166


In [32]:
# Equality of Odds

EQUALITY_OF_ODDS(X_val, y_val, lm_pred)

For recid = 0:

P_c(recid.hat = 1 | race = African American, recid = 0) =  0.2171945701357466
P_c(recid.hat = 1 | race = Caucasian, recid = 0) =  0.32051282051282054
Difference in odds of true recid = 0 is  = D_FPR = 0.10331825037707393



For recid = 1:

P_c(recid.hat = 1 | race = African American, recid = 1) =  0.5630630630630631
P_c(recid.hat = 1 | race = Caucasian, recid = 1) =  0.6532258064516129
Difference in odds of true recid = 1 is = D_TPR = 0.0901627433885498


## Local Preferential Sampling

In this algorithm, we take in a dataset (train) and return a modified dataset of same size.

In [36]:
pd.options.mode.chained_assignment = None
recomp_train = pd.DataFrame()

# for each partition (explanatory variable)
for X_ei in PARTITION(train):
    
    print("start partition")
    X_ei_copy = X_ei.copy()
    print("X_ei shape:", X_ei_copy.shape)
    
    # learn a ranker Hi : Xi -> Yi
    ranker_model = LogisticRegression(random_state=0).fit(X_ei[X_ALL], X_ei[Y])
    
    # Calculate half delta (AA: S_i = 1, AA: S_i = 0)
    half_delta_afam = DELTA(train, X_ei, 1) // 2
    half_delta_cauc = DELTA(train, X_ei, 0) // 2
    print("Half Delta(AA):", half_delta_afam)
    print("Half Delta(Cauc):", half_delta_cauc)
    
    # store indicies
    afam_index = [i for (i, v) in zip(list(range(X_ei.shape[0])), list(X_ei[S] == 1)) if v]
    c_index = [i for (i, v) in zip(list(range(X_ei.shape[0])), list(X_ei[S] == 0)) if v]
    print("Total AAs:", len(afam_index))
    print("Total Cs:", len(c_index))
    
    # get subset of data to work with
    afam = X_ei[X_ei[S] == 1].copy()
    c = X_ei[X_ei[S] == 0].copy()
    print("afam dataset shape:", afam.shape)
    print("c dataset shape:", c.shape)
    
    # rank AA
    afam.reset_index(drop=True, inplace=True)
    rank = pd.DataFrame(ranker_model.decision_function(afam[X_ALL]), columns = ['rank'])
    afam_with_rank = pd.concat([afam, rank], axis=1)
    
    # rank C
    c.reset_index(drop=True, inplace=True)
    rank = pd.DataFrame(ranker_model.decision_function(c[X_ALL]), columns = ['rank'])
    c_with_rank = pd.concat([c, rank], axis=1)
    
    # sort values, reset indices
    afam_with_rank = afam_with_rank.sort_values(['rank'])
    afam_with_rank.reset_index(drop = True, inplace = True)
    c_with_rank = c_with_rank.sort_values(['rank'])
    c_with_rank.reset_index(drop = True, inplace = True)
    
    ######## Modify AA data - find rows to delete/duplicate; decision boundary is 0 #####
    recid = sum(afam_with_rank['rank'] > 0)
    no_recid = sum(afam_with_rank['rank'] < 0)    
    total = len(afam_with_rank)
    
    # make copy of recids and no_recids
    # compas = compas[compas['days_b_screening_arrest'] >= -30]
    cleaned_recid = afam_with_rank[afam_with_rank['rank'] > 0]
    cleaned_no_recid = afam_with_rank[afam_with_rank['rank'] < 0]
    
    # delete first 1/2 delta values from recid
    N = half_delta_afam
    print("N:", N)
    print("rows in cleaned_recid before:", cleaned_recid.shape)
    cleaned_recid.drop(index=cleaned_recid.index[:N], axis=0, inplace=True)
    print("rows in cleaned_recid after:", cleaned_recid.shape)
    
    # flip order, then duplicate first 1/2 delta values from no_recid
    #print("cleaned_no_recid before:", cleaned_no_recid)
    cleaned_no_recid = cleaned_no_recid.sort_values(by='rank', ascending=False)
    #print("cleaned_no_recid after:", cleaned_no_recid)
    print("N:", N)
    print("rows in cleaned_no_recid before:", cleaned_no_recid.shape)
    cleaned_no_recid = cleaned_no_recid.append(cleaned_no_recid[0:N])
    print("rows in cleaned_no_recid after:", cleaned_no_recid.shape)
    
    # combine 
    total_AA = pd.concat([cleaned_recid, cleaned_no_recid])
    print("size of final A:", total_AA.shape)
    
    ########## Modify C data ############
    # Find rows to delete/duplicate; decision boundary is 0; opposite code as above
    recid = sum(c_with_rank['rank'] < 0)
    no_recid = sum(c_with_rank['rank'] > 0)    
    total = len(c_with_rank)
    
    # make copy of recids and no_recids
    cleaned_recid = c_with_rank[c_with_rank['rank'] < 0]
    cleaned_no_recid = c_with_rank[c_with_rank['rank'] > 0]
    
    # delete first 1/2 delta values from recid
    M = half_delta_cauc
    print("M:", M)
    print("rows in cleaned_recid before:", cleaned_recid.shape)
    cleaned_recid.drop(index=cleaned_recid.index[:M], axis=0, inplace=True)
    print("rows in cleaned_recid after:", cleaned_recid.shape)
    
    # flip order, then duplicate first 1/2 delta values from no_recid
    #print("cleaned_no_recid before:", cleaned_no_recid)
    cleaned_no_recid = cleaned_no_recid.sort_values(by='rank', ascending=False)
    #print("cleaned_no_recid after:", cleaned_no_recid)
    print("M:", M)
    print("rows in cleaned_no_recid before:", cleaned_no_recid.shape)
    cleaned_no_recid = cleaned_no_recid.append(cleaned_no_recid[0:M])
    print("rows in cleaned_no_recid after:", cleaned_no_recid.shape)
    
    # combine 
    total_C = pd.concat([cleaned_recid, cleaned_no_recid])
    print("size of final C:", total_C.shape)
    print("end partition")
    
    # combine both datasets
    recomp_train = recomp_train.append(total_AA)
    recomp_train = recomp_train.append(total_C)
    recomp_train = recomp_train.drop('rank', axis=1)
    
    print("size of train:", train.shape)
    print("size of recomp:", recomp_train.shape)
    

start partition
X_ei shape: (1245, 8)
Half Delta(AA): 22
Half Delta(Cauc): 19
Total AAs: 664
Total Cs: 581
afam dataset shape: (664, 8)
c dataset shape: (581, 8)
N: 22
rows in cleaned_recid before: (253, 9)
rows in cleaned_recid after: (231, 9)
N: 22
rows in cleaned_no_recid before: (411, 9)
rows in cleaned_no_recid after: (433, 9)
size of final A: (664, 9)
M: 19
rows in cleaned_recid before: (480, 9)
rows in cleaned_recid after: (461, 9)
M: 19
rows in cleaned_no_recid before: (101, 9)
rows in cleaned_no_recid after: (120, 9)
size of final C: (581, 9)
end partition
size of train: (3610, 8)
size of recomp: (1245, 8)
start partition
X_ei shape: (2365, 8)
Half Delta(AA): 49
Half Delta(Cauc): 28
Total AAs: 1504
Total Cs: 861
afam dataset shape: (1504, 8)
c dataset shape: (861, 8)
N: 49
rows in cleaned_recid before: (897, 9)
rows in cleaned_recid after: (848, 9)
N: 49
rows in cleaned_no_recid before: (607, 9)
rows in cleaned_no_recid after: (656, 9)
size of final A: (1504, 9)
M: 28
rows in 

## Evaluation for LPS Algorithm

In [38]:
recomp_train.head

<bound method NDFrame.head of      sex  age  race  decile_score  priors_count  c_charge_degree  \
433    0   28     1             4             2                0   
434    0   26     1             5             2                0   
435    0   34     1             7             0                0   
436    0   22     1             6             0                0   
437    0   38     1             4             5                0   
..   ...  ...   ...           ...           ...              ...   
837    1   34     0             9            15                1   
836    0   23     0             7            10                1   
835    0   45     0             6            21                1   
834    0   30     0             9             9                1   
833    0   24     0            10             6                1   

     two_year_recid       los  
433               0  4.976734  
434               0  3.912023  
435               1  6.549651  
436               0  4.24

In [45]:
recomp_X_train = recomp_train[X_ALL]
recomp_Y_train = recomp_train[Y]
print("size of recomp_X_train:", recomp_X_train.shape)
print("size of recomp_Y_train:", recomp_Y_train.shape)

clf_LPS = LogisticRegression(random_state=0).fit(recomp_X_train, recomp_Y_train)
LPS_pred = clf_LPS.predict(X_val[X_ALL])
clf_LPS.score(X_val[X_ALL], y_val)

size of recomp_X_train: (3610, 7)
size of recomp_Y_train: (3610,)


0.686030428769018

In [46]:
print(classification_report(y_val, clf_LPS.predict(X_val[X_ALL])))

              precision    recall  f1-score   support

           0       0.68      0.75      0.71       377
           1       0.69      0.62      0.65       346

    accuracy                           0.69       723
   macro avg       0.69      0.68      0.68       723
weighted avg       0.69      0.69      0.68       723



In [47]:
PARITY(X_val, LPS_pred)

P_c(recid = 1 | race = African American) = 0.5033860045146726
P_c(recid = 1 | race = Caucasian) = 0.3142857142857143
Parity = 0.18910029022895836


0.18910029022895836

In [48]:
CALIBRATION(X_val, y_val, LPS_pred)

P_c(recid predicted correctly | race = African American) = 0.6862302483069977
P_c(recid predicted correctly | race = Caucasian) = 0.6857142857142857
Calibration = 0.000515962592712027


In [49]:
EQUALITY_OF_ODDS(X_val, y_val, LPS_pred)

For recid = 0:

P_c(recid.hat = 1 | race = African American, recid = 0) =  0.3167420814479638
P_c(recid.hat = 1 | race = Caucasian, recid = 0) =  0.16666666666666666
Difference in odds of true recid = 0 is  = D_FPR = 0.15007541478129713



For recid = 1:

P_c(recid.hat = 1 | race = African American, recid = 1) =  0.6891891891891891
P_c(recid.hat = 1 | race = Caucasian, recid = 1) =  0.5
Difference in odds of true recid = 1 is = D_TPR = 0.18918918918918914


In [50]:
# D_FNR

D_FNR(X_val, y_val, LPS_pred)

Difference in False Negative Rates
For recid = 1:

P_c(recid.hat = 0 | race = African American, recid = 1) =  0.3108108108108108
P_c(recid.hat = 0 | race = Caucasian, recid = 1) =  0.5
D_FNR = 0.1891891891891892
